Install the following libraries before running cells
pip install transformers
pip install torch
pip install datasets
pip install evaluate

In [1]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import TrainingArguments, Trainer, DistilBertTokenizer, DistilBertForSequenceClassification
import evaluate
import numpy as np;
from transformers import DataCollatorWithPadding 
import torch

C:\Users\Arfaat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_dict = load_dataset("csv", data_files={"train": "IMDB Dataset.csv"})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 50000
    })
})


In [4]:
# For binary classification, use only two classes
model_path = "distilbert-base-uncased"  # Use a smaller model for faster processing

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Update the label mappings for binary classification
id2label = {
    0: "negative",  # Negative class
    1: "positive"   # Positive class
}

label2id = {
    "negative": 0,
    "positive": 1
}


# Load the model with the updated number of classes (2 for binary classification)
model = DistilBertForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2,  # For binary classification, use 2 labels
    id2label=id2label,
    label2id=label2id
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Freeze all base model parameters except the classification head
for name, param in model.distilbert.named_parameters():
    param.requires_grad = False

# If you want to unfreeze the classification head, you can do that as follows:
for param in model.classifier.parameters():
    param.requires_grad = True


In [6]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

# Preprocess function to tokenize the text and handle labels
def preprocess_function(examples, tokenizer):
    # Use the 'review' column for the text input (instead of 'reviews.text')
    text = [str(review) for review in examples["review"]]  # Extract review text from the dataset
    
    # Convert 'sentiment' to binary labels: 1 for 'positive' and 0 for 'negative'
    labels = [1 if sentiment == 'positive' else 0 for sentiment in examples["sentiment"]]  # Positive = 1, Negative = 0
    
    # Tokenize and pad/truncate the text (Customize max_length as needed)
    tokenized_output = tokenizer(text, truncation=True, padding="max_length", max_length=128)  # Set max_length to a suitable value
    
    # Add labels to the tokenized output
    tokenized_output["labels"] = labels
    
    return tokenized_output

# Convert the dataset into a pandas DataFrame to facilitate splitting
train_data_list = dataset_dict["train"].to_pandas()

# Convert 'sentiment' to binary labels: 1 for 'positive', 0 for 'negative'
train_data_list['label'] = train_data_list['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# Separate the data into 'positive' and 'negative' classes
df_positive = train_data_list[train_data_list['label'] == 1]
df_negative = train_data_list[train_data_list['label'] == 0]

# Print the sizes of each class (No need to sample since the dataset is already balanced)
print("Positive Reviews Length: ", len(df_positive))
print("Negative Reviews Length: ", len(df_negative))

# Combine the balanced data (no need for resampling)
df_balanced = pd.concat([df_positive, df_negative])

# Shuffle the combined dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert back to Hugging Face Dataset format after shuffling
dataset_dict["train"] = Dataset.from_pandas(df_balanced)

# Perform stratified splitting using sklearn to create train and test splits
train_data, test_data = train_test_split(
    df_balanced, 
    test_size=0.2, 
    stratify=df_balanced['label']  # Ensures balanced class distribution in train and test
)

# Convert back to Hugging Face Dataset format after splitting
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

# Apply preprocessing to both train and test datasets
train_data = train_data.map(preprocess_function, batched=True, num_proc=1, fn_kwargs={"tokenizer": tokenizer}, keep_in_memory=False)
test_data = test_data.map(preprocess_function, batched=True, num_proc=1, fn_kwargs={"tokenizer": tokenizer}, keep_in_memory=False)


print("train data:", train_data)
print("test data:", test_data)
# Now, train_data and test_data are ready for use in the model


Positive Reviews Length:  25000
Negative Reviews Length:  25000


Map: 100%|██████████| 10000/10000 [00:33<00:00, 300.57 examples/s]

train data: Dataset({
    features: ['review', 'sentiment', 'label', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 40000
})
test data: Dataset({
    features: ['review', 'sentiment', 'label', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})


In [6]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Get the predicted class labels (0 or 1)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Get predicted class probabilities (for AUC calculation)
    predicted_probs = predictions[:, 1]  # Probabilities for the positive class (1)

    # Calculate AUC using roc_auc_score for binary classification
    auc = roc_auc_score(labels, predicted_probs)  # For binary, no need for 'multi_class' option
    
    # Calculate other metrics
    accuracy = accuracy_score(labels, predicted_classes)
    precision = precision_score(labels, predicted_classes)
    recall = recall_score(labels, predicted_classes)
    f1 = f1_score(labels, predicted_classes)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }




In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save at the end of each epoch
    learning_rate=2e-5,  # Adjust learning rate
    per_device_train_batch_size=16,  # Increase batch size for faster processing
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # Reduce number of epochs
    logging_dir='./logs',
    logging_steps=50,
    save_steps=500,  # You may remove this as 'save_strategy' is set to 'epoch'
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed-precision training
    gradient_accumulation_steps=2,  # For larger effective batch size
    report_to="none"  # Avoid sending reports to external services
)


C:\Users\Arfaat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,  # Your binary classification train dataset
    eval_dataset=test_data,    # Your binary classification test dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Ensure you're using the updated metrics function
)

trainer.train()


C:\Users\Arfaat\AppData\Local\Temp\ipykernel_3124\3503835185.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  2%|▏         | 50/2500 [01:46<1:29:02,  2.18s/it]

{'loss': 0.6882, 'grad_norm': 1.1521048545837402, 'learning_rate': 1.9600000000000002e-05, 'epoch': 0.04}


  4%|▍         | 100/2500 [03:35<1:27:31,  2.19s/it]

{'loss': 0.6719, 'grad_norm': 1.4060633182525635, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.08}


  6%|▌         | 150/2500 [05:57<2:52:23,  4.40s/it]

...

                                                     
100%|██████████| 2500/2500 [1:56:10<00:00,  2.92s/it]

{'eval_loss': 0.4728476107120514, 'eval_accuracy': 0.7832, 'eval_precision': 0.7813990461049285, 'eval_recall': 0.7864, 'eval_f1': 0.7838915470494418, 'eval_auc': 0.8619363600000001, 'eval_runtime': 685.5477, 'eval_samples_per_second': 14.587, 'eval_steps_per_second': 0.912, 'epoch': 2.0}


100%|██████████| 2500/2500 [1:56:11<00:00,  2.79s/it]

{'train_runtime': 6971.371, 'train_samples_per_second': 11.476, 'train_steps_per_second': 0.359, 'train_loss': 0.5230634147644043, 'epoch': 2.0}


TrainOutput(global_step=2500, training_loss=0.5230634147644043, metrics={'train_runtime': 6971.371, 'train_samples_per_second': 11.476, 'train_steps_per_second': 0.359, 'total_flos': 2649347973120000.0, 'train_loss': 0.5230634147644043, 'epoch': 2.0})

In [10]:
model.save_pretrained("results")
tokenizer.save_pretrained("results")

('results\\tokenizer_config.json',
 'results\\special_tokens_map.json',
 'results\\vocab.txt',
 'results\\added_tokens.json')